# DSB Model Test

* V22 
* V22 + add
* V23 
* V23 + add

In [1]:
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import gc

import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import warnings
warnings.filterwarnings("ignore")

import tqdm
from tqdm import tqdm_notebook
import os

#Print control function
print('Set print')
def print_expand():
    pd.set_option('display.max_columns', None)  # or 1000
    pd.set_option('display.max_rows', 100)  # or 1000
    pd.set_option('display.max_colwidth', -1)  # or 199
    
def print_basic():
    pd.set_option('display.max_columns', 30)  # or 1000
    pd.set_option('display.max_rows', 30)  # or 1000
    pd.set_option('display.max_colwidth', 50)  # or 199

Set print


## Load train data

In [2]:
train_data = pd.read_csv('../input/new-train/new_train_v24_concat.csv')

In [3]:
print_expand()
print(len(train_data), len(train_data.columns))
train_data.head(5)

19708 344


,installation_id,game_session,title,game_time,time_gap,total_time,time_percentage,Clip_time_per,Activity_time_per,Game_time_per,Assessment_time_per,session_count,Activity,Assessment,Clip,Game,between_3xxx,code_4xx_time,mushroom_p_count,bird_p_count,cauldron_p_count,chest_p_count,cart_p_count,mushroom_p_per,bird_p_per,cauldron_p_per,chest_p_per,cart_p_per,2000,2010,2020,2025,2030,2035,2040,2050,2060,2070,2075,2080,2081,2083,3010,3020,3021,4010,4020,4021,4022,4025,4030,4031,4035,4040,4045,4050,4070,4080,4090,4095,4100,4110,4220,4230,4235,5000,5010,num_correct,num_incorrect,acc,accuracy_group,Scrub-A-Dub_count,Scrub-A-Dub_duration_x,Scrub-A-Dub_final,Scrub-A-Dub_final_duration_mean,Scrub-A-Dub_final_T_mean,Scrub-A-Dub_final_F_mean,Scrub-A-Dub_zero_T,Scrub-A-Dub_zero_F,Scrub-A-Dub_zero_duration,Scrub-A-Dub_process_T,Scrub-A-Dub_process_F,Scrub-A-Dub_process_Acc,All Star Sorting_count,All Star Sorting_duration_x,All Star Sorting_final,All Star Sorting_final_duration_mean,All Star Sorting_final_T_mean,All Star Sorting_final_F_mean,All Star Sorting_zero_T,All Star Sorting_zero_F,All Star Sorting_zero_duration,All Star Sorting_process_T,All Star Sorting_process_F,All Star Sorting_process_Acc,Air Show_count,Air Show_duration_x,Air Show_final,Air Show_final_duration_mean,Air Show_final_T_mean,Air Show_final_F_mean,Air Show_zero_T,Air Show_zero_F,Air Show_zero_duration,Air Show_process_T,Air Show_process_F,Air Show_process_Acc,Crystals Rule_count,Crystals Rule_duration_x,Crystals Rule_final,Crystals Rule_final_duration_mean,Crystals Rule_final_T_mean,Crystals Rule_final_F_mean,Crystals Rule_zero_T,Crystals Rule_zero_F,Crystals Rule_zero_duration,Crystals Rule_process_T,Crystals Rule_process_F,Crystals Rule_process_Acc,Dino Drink_count,Dino Drink_duration_x,Dino Drink_final,Dino Drink_final_duration_mean,Dino Drink_final_T_mean,Dino Drink_final_F_mean,Dino Drink_zero_T,Dino Drink_zero_F,Dino Drink_zero_duration,Dino Drink_process_T,Dino Drink_process_F,Dino Drink_process_Acc,Bubble Bath_count,Bubble Bath_duration_x,Bubble Bath_final,Bubble Bath_final_duration_mean,Bubble Bath_final_T_mean,Bubble Bath_final_F_mean,Bubble Bath_zero_T,Bubble Bath_zero_F,Bubble Bath_zero_duration,Bubble Bath_process_T,Bubble Bath_process_F,Bubble Bath_process_Acc,Dino Dive_count,Dino Dive_duration_x,Dino Dive_final,Dino Dive_final_duration_mean,Dino Dive_final_T_mean,Dino Dive_final_F_mean,Dino Dive_zero_T,Dino Dive_zero_F,Dino Dive_zero_duration,Dino Dive_process_T,Dino Dive_process_F,Dino Dive_process_Acc,Chow Time_count,Chow Time_duration_x,Chow Time_final,Chow Time_final_duration_mean,Chow Time_final_T_mean,Chow Time_final_F_mean,Chow Time_zero_T,Chow Time_zero_F,Chow Time_zero_duration,Chow Time_process_T,Chow Time_process_F,Chow Time_process_Acc,Pan Balance_count,Pan Balance_duration_x,Pan Balance_final,Pan Balance_final_duration_mean,Pan Balance_final_T_mean,Pan Balance_final_F_mean,Pan Balance_zero_T,Pan Balance_zero_F,Pan Balance_zero_duration,Pan Balance_process_T,Pan Balance_process_F,Pan Balance_process_Acc,Happy Camel_count,Happy Camel_duration_x,Happy Camel_final,Happy Camel_final_duration_mean,Happy Camel_final_T_mean,Happy Camel_final_F_mean,Happy Camel_zero_T,Happy Camel_zero_F,Happy Camel_zero_duration,Happy Camel_process_T,Happy Camel_process_F,Happy Camel_process_Acc,Leaf Leader_count,Leaf Leader_duration_x,Leaf Leader_final,Leaf Leader_final_duration_mean,Leaf Leader_final_T_mean,Leaf Leader_final_F_mean,Leaf Leader_zero_T,Leaf Leader_zero_F,Leaf Leader_zero_duration,Leaf Leader_process_T,Leaf Leader_process_F,Leaf Leader_process_Acc,Welcome to Lost Lagoon!,Magma Peak - Level 1,Slop Problem,Tree Top City - Level 1,Ordering Spheres,Costume Box,12 Monkeys,Tree Top City - Level 2,Pirate's Tale,Treasure Map,Tree Top City - Level 3,Rulers,Magma Peak - Level 2,Crystal Caves - Level 1,Balancing Act,Crystal Caves - Level 2,Crystal Caves - Level 3,Lifting Heavy Things,Honey Cake,"Heavy, Heavier, Heaviest",Sandcastle Builder (Activity)_code_2x,Sand

In [4]:
def preprocess(train_data):
        ## Null values
    for i in ['time_percentage', 'Clip_time_per', 'Activity_time_per', 'Game_time_per', 'code_4xx_time']:
    
        train_data[i].fillna(0, inplace=True)

    for i in ['Scrub-A-Dub_process_Acc', 'All Star Sorting_process_Acc',
           'Air Show_process_Acc', 'Crystals Rule_process_Acc',
           'Dino Drink_process_Acc', 'Bubble Bath_process_Acc',
           'Dino Dive_process_Acc', 'Chow Time_process_Acc',
           'Pan Balance_process_Acc', 'Happy Camel_process_Acc',
           'Leaf Leader_process_Acc']:
        
        train_data[i].fillna(0, inplace=True)
        
            ### Assessment dictionary
    assess_list = {'Mushroom Sorter (Assessment)': 0, 
                   'Bird Measurer (Assessment)': 1, 
                   'Cauldron Filler (Assessment)': 2, 
                   'Chest Sorter (Assessment)': 3,
                   'Cart Balancer (Assessment)': 4}

    del_clip = ['Welcome to Lost Lagoon!', 'Magma Peak - Level 1', 'Slop Problem',
           'Tree Top City - Level 1', 'Ordering Spheres', 'Costume Box',
           '12 Monkeys', 'Tree Top City - Level 2',"Pirate's Tale",
           'Treasure Map', 'Tree Top City - Level 3', 'Rulers',
           'Magma Peak - Level 2', 'Crystal Caves - Level 1', 'Balancing Act',
           'Crystal Caves - Level 2', 'Crystal Caves - Level 3',
           'Lifting Heavy Things', 'Honey Cake', 'Heavy, Heavier, Heaviest']

        
    ### 1. Assessment encoding
    train_data['title'] = train_data['title'].map(assess_list)
    
    ### ** Group ###
    group_train = train_data['title'].values
    
    ### 2. Make title column to dummy columns
    train_data = pd.concat([train_data, pd.get_dummies(train_data['title'], prefix='Assess_')], axis=1)
    
    
    ### 3. Target column
    target = train_data['accuracy_group'].copy()

    ### 4. Drop columns
    train_data.drop(columns=(['installation_id', 'game_session', 'title',
                              'num_correct', 'num_incorrect', 'acc', 'accuracy_group']), inplace=True)
    
    
    ### 5. Drop previous Game features
    train_data.drop(columns=del_clip, inplace=True, errors='ignore')
    
    return train_data, target, group_train

In [5]:
train_data, target, group_tr= preprocess(train_data)

In [6]:
train_data.isnull().sum().sum()#, test_data.isnull().sum().sum()

0

In [7]:
print_expand()
#print(len(train_data.columns), len(test_data.columns))
display(train_data.head())
#display(test_data.head())

,game_time,time_gap,total_time,time_percentage,Clip_time_per,Activity_time_per,Game_time_per,Assessment_time_per,session_count,Activity,Assessment,Clip,Game,between_3xxx,code_4xx_time,mushroom_p_count,bird_p_count,cauldron_p_count,chest_p_count,cart_p_count,mushroom_p_per,bird_p_per,cauldron_p_per,chest_p_per,cart_p_per,2000,2010,2020,2025,2030,2035,2040,2050,2060,2070,2075,2080,2081,2083,3010,3020,3021,4010,4020,4021,4022,4025,4030,4031,4035,4040,4045,4050,4070,4080,4090,4095,4100,4110,4220,4230,4235,5000,5010,Scrub-A-Dub_count,Scrub-A-Dub_duration_x,Scrub-A-Dub_final,Scrub-A-Dub_final_duration_mean,Scrub-A-Dub_final_T_mean,Scrub-A-Dub_final_F_mean,Scrub-A-Dub_zero_T,Scrub-A-Dub_zero_F,Scrub-A-Dub_zero_duration,Scrub-A-Dub_process_T,Scrub-A-Dub_process_F,Scrub-A-Dub_process_Acc,All Star Sorting_count,All Star Sorting_duration_x,All Star Sorting_final,All Star Sorting_final_duration_mean,All Star Sorting_final_T_mean,All Star Sorting_final_F_mean,All Star Sorting_zero_T,All Star Sorting_zero_F,All Star Sorting_zero_duration,All Star Sorting_process_T,All Star Sorting_process_F,All Star Sorting_process_Acc,Air Show_count,Air Show_duration_x,Air Show_final,Air Show_final_duration_mean,Air Show_final_T_mean,Air Show_final_F_mean,Air Show_zero_T,Air Show_zero_F,Air Show_zero_duration,Air Show_process_T,Air Show_process_F,Air Show_process_Acc,Crystals Rule_count,Crystals Rule_duration_x,Crystals Rule_final,Crystals Rule_final_duration_mean,Crystals Rule_final_T_mean,Crystals Rule_final_F_mean,Crystals Rule_zero_T,Crystals Rule_zero_F,Crystals Rule_zero_duration,Crystals Rule_process_T,Crystals Rule_process_F,Crystals Rule_process_Acc,Dino Drink_count,Dino Drink_duration_x,Dino Drink_final,Dino Drink_final_duration_mean,Dino Drink_final_T_mean,Dino Drink_final_F_mean,Dino Drink_zero_T,Dino Drink_zero_F,Dino Drink_zero_duration,Dino Drink_process_T,Dino Drink_process_F,Dino Drink_process_Acc,Bubble Bath_count,Bubble Bath_duration_x,Bubble Bath_final,Bubble Bath_final_duration_mean,Bubble Bath_final_T_mean,Bubble Bath_final_F_mean,Bubble Bath_zero_T,Bubble Bath_zero_F,Bubble Bath_zero_duration,Bubble Bath_process_T,Bubble Bath_process_F,Bubble Bath_process_Acc,Dino Dive_count,Dino Dive_duration_x,Dino Dive_final,Dino Dive_final_duration_mean,Dino Dive_final_T_mean,Dino Dive_final_F_mean,Dino Dive_zero_T,Dino Dive_zero_F,Dino Dive_zero_duration,Dino Dive_process_T,Dino Dive_process_F,Dino Dive_process_Acc,Chow Time_count,Chow Time_duration_x,Chow Time_final,Chow Time_final_duration_mean,Chow Time_final_T_mean,Chow Time_final_F_mean,Chow Time_zero_T,Chow Time_zero_F,Chow Time_zero_duration,Chow Time_process_T,Chow Time_process_F,Chow Time_process_Acc,Pan Balance_count,Pan Balance_duration_x,Pan Balance_final,Pan Balance_final_duration_mean,Pan Balance_final_T_mean,Pan Balance_final_F_mean,Pan Balance_zero_T,Pan Balance_zero_F,Pan Balance_zero_duration,Pan Balance_process_T,Pan Balance_process_F,Pan Balance_process_Acc,Happy Camel_count,Happy Camel_duration_x,Happy Camel_final,Happy Camel_final_duration_mean,Happy Camel_final_T_mean,Happy Camel_final_F_mean,Happy Camel_zero_T,Happy Camel_zero_F,Happy Camel_zero_duration,Happy Camel_process_T,Happy Camel_process_F,Happy Camel_process_Acc,Leaf Leader_count,Leaf Leader_duration_x,Leaf Leader_final,Leaf Leader_final_duration_mean,Leaf Leader_final_T_mean,Leaf Leader_final_F_mean,Leaf Leader_zero_T,Leaf Leader_zero_F,Leaf Leader_zero_duration,Leaf Leader_process_T,Leaf Leader_process_F,Leaf Leader_process_Acc,Sandcastle Builder (Activity)_code_2x,Sandcastle Builder (Activity)_code_4t,Sandcastle Builder (Activity)_code_4f,Fireworks (Activity)_code_2x,Fireworks (Activity)_code_4t,Fireworks (Activity)_code_4f,Flower Waterer (Activity)_code_2x,Flower Waterer (Activity)_code_4t,Flower Waterer (Activity)_code_4f,Bug Measurer (Activity)_code_2x,Bug Measurer (Activity)_code_4t,Bug Measurer (Activity)_code_4f,Watering Hole (Activity)_code_2x,Watering Hole (Activity)_code_4t,Watering Hole (Activity)_co

## LGb_model 

https://www.kaggle.com/skylord/data-science-bowl-2019-regression-to-convert-lb

In [8]:
with open('../input/new-train/val_dict_add2.pkl', 'rb') as f:
    val_dict = pickle.load(f)

In [9]:
# thanks to Andrew
def get_label(vector):
    vector[vector <= 1.12232214] = 0
    vector[np.where(np.logical_and(vector > 1.12232214, vector <= 1.73925866))] = 1
    vector[np.where(np.logical_and(vector > 1.73925866, vector <= 2.22506454))] = 2
    vector[vector > 2.22506454] = 3
    return vector

def cohen_kappa(y_pred, y):
    y = y.get_label().astype("int")
    y_pred = get_label(y_pred)
    loss = cohen_kappa_score(y, y_pred, weights = 'quadratic')
    return "cohen kappa score", loss, True

def run_lgb(reduce_train, target, val_dict, ps=False):
    
    #Set params
    params = {'verbose': 100,
              'learning_rate': 0.010514633017309072,
              'metric': 'rmse',
              'bagging_freq': 3,
              'boosting_type': 'gbdt',
              'eval_metric': 'cappa',
              'lambda_l1': 4.8999704874480745,
              'colsample_bytree': 0.4236269531042225,
              'early_stopping_rounds': 100,
              'max_depth': 12,
              'lambda_l2': 0.054084652510602016,
              'bagging_fraction': 0.7931423220563563,
              'n_jobs': -1,
              'n_estimators': 2000,
              'objective': 'regression',
              'seed': 42}
    
    if ps:
        print('Parameter change')
        params['bagging_fraction']=ps['bagging_fraction']
        #params['learning_rate']=ps['learning_rate']
        #params['feature_fraction']=ps['feature_fraction']
        params['lambda_l1']=ps['lambda_l1']
        params['lambda_l2']=ps['lambda_l2']
        params['max_depth']=int(ps['max_depth'])
        #params['n_estimators']=int(ps['n_estimators'])
        #params['min_data_in_leaf']=int(ps['min_data_in_leaf'])
        params['bagging_freq']=int(ps['bagging_freq'])
        #params['early_stopping_rounds']=int(ps['early_stopping_rounds'])
        params['num_leaves'] = int(ps['num_leaves'])
        #params['drop_rate'] : params['drop_rate']
        
    
    X = reduce_train.values
    y = target.values
    features = reduce_train.columns
    
    # define a GroupKFold strategy because we are predicting unknown installation_ids
    #kf = KFold(n_splits=5, shuffle=True, random_state=2019)
    #target = 'accuracy_group'
    oof_pred = np.zeros(len(reduce_train))
    train_pred = np.zeros(len(reduce_train))
    important_features = pd.DataFrame()
    
    # train a baseline model and record the cohen cappa score as our best score
    #for n_fold, (tr_idx, val_idx) in enumerate(kf.split(reduce_train)):
    for n_fold in range(5):
        print("="*50)
        print('Training Start, fold : {}'.format(n_fold+1))
        print("="*50)
        val_idx = val_dict[n_fold]
        tr_idx = train_data.index[reduce_train.index.isin(val_idx) == False]
        
        train_x, train_y = X[tr_idx], y[tr_idx]
        valid_x, valid_y = X[val_idx], y[val_idx]
        lgb_train = lgb.Dataset(train_x,train_y)
        lgb_valid = lgb.Dataset(valid_x,valid_y)
        
        model = lgb.train(params, lgb_train, num_boost_round = 100000,#, early_stopping_rounds = 100, 
                         valid_sets = [lgb_train, lgb_valid], verbose_eval = 500, feval = cohen_kappa)
        
        oof_pred[val_idx] = model.predict(valid_x)
        
        train_pred += model.predict(reduce_train) / 5
        
        #Save feature importance
        fold_importance = pd.DataFrame()
        fold_importance['features'] = features
        fold_importance['importance'] = model.feature_importance()
        important_features = pd.concat([important_features, fold_importance], axis = 1)
        #if n_fold == 2:
        #    break
        
    # calculate loss
    loss_score = cohen_kappa_score(target, get_label(oof_pred), weights = 'quadratic')
    print('Our oof cohen kappa score is :', loss_score)
    print(pd.Series(get_label(oof_pred)).value_counts(normalize = True))
    return loss_score

## Grid search

In [10]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events


In [11]:
def LGB_Beyes(bagging_fraction,
              #learning_rate,
              #feature_fraction,
              lambda_l1,
              lambda_l2,
              max_depth,
              #n_estimators,
              #min_data_in_leaf,
              bagging_freq,
              num_leaves,
              early_stopping_rounds,
              #drop_rate
             ):
    params={}
    params['bagging_fraction']=bagging_fraction
    #params['learning_rate']=learning_rate
    #params['feature_fraction']=feature_fraction
    params['num_leaves'] = num_leaves
    params['lambda_l1']=lambda_l1
    params['lambda_l2']=lambda_l2
    params['max_depth']=max_depth
    #params['n_estimators'] = n_estimators
    #params['min_data_in_leaf'] = min_data_in_leaf
    params['bagging_freq'] = bagging_freq
    params['early_stopping_rounds'] = early_stopping_rounds
    #params['drop_rate'] = drop_rate
    
    loss = run_lgb(train_data, target, val_dict, ps=params)
    print('kappa: ',loss)
    return loss

bounds_LGB = {
    'bagging_fraction': (0.5, 1),
    #'learning_rate': (0.003, 0.03),
    #'feature_fraction': (0.7, 1),
    'lambda_l1': (0, 5),
    'lambda_l2': (0, 5),
    'max_depth': (5, 12),
    'num_leaves' : (31, 300),
    #'n_estimators':(500, 3000),
    #'min_data_in_leaf':(20, 250),
    'bagging_freq':(3,10),
    'early_stopping_rounds':(100, 400)
    #'drop_rate' : (0.1, 0.3)
    #'scale_pos_weight' : (1, 1,2)
}

LGB_BO = BayesianOptimization(LGB_Beyes, bounds_LGB, random_state=1029)
logger = JSONLogger(path="logs.json")
LGB_BO.subscribe(Events.OPTMIZATION_STEP, logger)
import warnings
init_points = 10
n_iter = 16
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

Parameter change
Training Start, fold : 1
Training until validation scores don't improve for 100 rounds
[500]	training's rmse: 0.897289	training's cohen kappa score: 0.687815	valid_1's rmse: 0.980721	valid_1's cohen kappa score: 0.603812
Early stopping, best iteration is:
[532]	training's rmse: 0.892938	training's cohen kappa score: 0.691462	valid_1's rmse: 0.980045	valid_1's cohen kappa score: 0.605866
Training Start, fold : 2
Training until validation scores don't improve for 100 rounds
[500]	training's rmse: 0.897495	training's cohen kappa score: 0.684835	valid_1's rmse: 0.984356	valid_1's cohen kappa score: 0.595349
Early stopping, best iteration is:
[443]	training's rmse: 0.906367	training's cohen kappa score: 0.679378	valid_1's rmse: 0.986271	valid_1's cohen kappa score: 0.601454
Training Start, fold : 3
Training until validation scores don't improve for 100 rounds
[500]	training's rmse: 0.902916	training's cohen kappa score: 0.682115	valid_1's rmse: 0.962642	valid_1's cohen kapp

In [12]:
LGB_BO.max['params']

{'bagging_fraction': 0.5845620850690344,
 'bagging_freq': 8.711732836609187,
 'early_stopping_rounds': 273.93572247936913,
 'lambda_l1': 0.27892942339858784,
 'lambda_l2': 0.5922010574301761,
 'max_depth': 11.676293897239386,
 'num_leaves': 179.66680254812528}

In [13]:
print(LGB_BO.max)

{'target': 0.6120180552179686, 'params': {'bagging_fraction': 0.5845620850690344, 'bagging_freq': 8.711732836609187, 'early_stopping_rounds': 273.93572247936913, 'lambda_l1': 0.27892942339858784, 'lambda_l2': 0.5922010574301761, 'max_depth': 11.676293897239386, 'num_leaves': 179.66680254812528}}
